In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from IPython.display import display
from sklearn.metrics import auc
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('course_lead_scoring.csv')

In [3]:
df.head(5)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
df.lead_source = df.lead_source.fillna('NA')
df.industry = df.industry.fillna('NA')
df.annual_income = df.annual_income.fillna(0.0)
df.employment_status = df.employment_status.fillna('NA')
df.location = df.location.fillna('NA')

In [7]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [8]:
numerical  = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [10]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
len(df_train), len(df_test), len(df_val)

(876, 293, 293)

In [11]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [12]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [13]:
df_num = df_train[numerical]
df_num.head()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,2,95543.0,3,0.78
1,1,54924.0,6,0.39
2,2,77352.0,2,0.22
3,2,34600.0,2,0.31
4,0,43615.0,2,0.01


In [14]:
feature_auc_scores = {}

for feature in df_num.columns:
    auc_score = roc_auc_score(y_train, df_num[feature])
    feature_auc_scores[feature] = auc_score

sorted_auc_scores = sorted(feature_auc_scores.items(), key=lambda x: x[1], reverse=True)

for feature, auc in sorted_auc_scores:
    print(f"Feature: {feature}, AUC: {auc:.4f}")

Feature: number_of_courses_viewed, AUC: 0.7636
Feature: interaction_count, AUC: 0.7383
Feature: lead_score, AUC: 0.6145
Feature: annual_income, AUC: 0.5520


In [15]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [16]:
dv, model = train(df_train, y_train)

In [17]:
y_pred = predict(df_val, dv, model)

In [18]:
roc_auc = roc_auc_score(y_val, y_pred)
round(roc_auc, 3)

0.817